# Figure 2: MS2/Identification visualization

## Library imports

In [18]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

import utils
import alphatims.bruker

## Fig 2 A: MS2 DDA spectra 

For this figure we used the [PXD012867 PRIDE project](https://www.ebi.ac.uk/pride/archive/projects/PXD012867) as example data.

In [2]:
sample_path = '.../Data/PXD012867_yeast_project/20190124_QX3_JuSc_SA_JS7_1_wt_4h_1.raw'

In [3]:
data = utils.load_thermo_raw(sample_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 139295/139295 [02:11<00:00, 1061.49it/s]


In [4]:
# this plotting function is taken from the AlphaViz package (https://github.com/MannLabs/alphaviz) and modified
def plot_mass_spectra(
    mz_values: np.ndarray,
    intensity_values: np.ndarray,
    title: str,
    spectrum_color: str = 'grey',
    template: str = "plotly_white",
    spectrum_line_width: float = 1.5,
    height: int = 520
):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=mz_values,
            y=intensity_values,
            mode='markers',
            marker=dict(color=spectrum_color, size=1),
            hovertemplate='<b>m/z:</b> %{x};<br><b>Intensity:</b> %{y}.',
            name='',
            showlegend=False
        )
    )

    # Use the 'shapes' attribute from the layout to draw the vertical lines
    fig.update_layout(
        template=template,
        shapes=[
            dict(
                type='line',
                xref='x',
                yref='y',
                x0=mz_values[i],
                y0=0,
                x1=mz_values[i],
                y1=intensity_values[i],
                line=dict(
                    color = spectrum_color,
                    width=spectrum_line_width
                )
            ) for i, val in enumerate(zip(mz_values, intensity_values))
        ],
        xaxis=dict(
            title='m/z, Th',
        ),
        yaxis=dict(
            title='Intensity',
        ),
        legend=dict(
            orientation="h",
            x=1,
            xanchor="right",
            yanchor="bottom",
            y=1.01
        ),
        hovermode="closest",
        height=height,
        title=dict(
            text=title,
            font=dict(
                size=16,
            ),
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
    )

    return fig

In [7]:
df_ms2 = pd.DataFrame({'scan': data['scan_list_ms2'], 
                    'RT': data['rt_list_ms2'],
                    'prec_mono_mz': data['mono_mzs2'],
                    'prec_charge': data['charge_ms2'],
                    'mz_values': data['mass_list_ms2'],
                   'intensity_values': data['int_list_ms2'],
                    'order': 'ms2'})
df_ms2[df_ms2.RT > 50].head()

,scan,RT,prec_mono_mz,prec_charge,mz_values,intensity_values,order
57334,61885,50.001900,590.2973,2,"[100.41511535644531, 101.07136535644531, 101.1...","[7199, 12615, 6445, 21390, 50917, 6285, 11642,...",ms2
57335,61886,50.002591,479.4426,2,"[101.07112121582031, 102.05511474609375, 103.0...","[24542, 28179, 15409, 96100, 8020, 13094, 2720...",ms2
57336,61887,50.003293,732.3805,2,"[101.0713119506836, 102.05506896972656, 109.51...","[14165, 12270, 6441, 34719, 8217, 15476, 7853,...",ms2
57337,61888,50.004004,625.6472,3,"[102.05464172363281, 110.07127380371094, 120.0...","[12502, 37669, 29084, 8254, 8492, 53982, 32433...",ms2
57338,61889,50.004702,517.2747,2,"[101.07125854492188, 102.05510711669922, 110.0...","[17768, 35090, 38741, 5938, 42354, 6035, 23082...",ms2


In [66]:
# let's plot the MS2 spectra acquired at 50.0019 min for the precursor m/z = 590.2973
plot_mass_spectra(
    mz_values=df_ms2[df_ms2.scan == 61885].mz_values.values[0], 
    intensity_values=df_ms2[df_ms2.scan == 61885].intensity_values.values[0],
    title=f"MS2 DDA spectrum for precursor m/z={df_ms2[df_ms2.scan == 61885].prec_mono_mz.values[0]} and charge={df_ms2[df_ms2.scan == 61885].prec_charge.values[0]}."
).show(config=utils.config)

## Fig 2 B: MS2 DIA spectra 

For this figure we used the Bruker raw file is used from the [Project PXD017703 from ProteomeXchange](https://www.ebi.ac.uk/pride/archive/projects/PXD017703). as example data.

To read the raw TIMS-TOF data we use a recently published [AlphaTims package](https://github.com/MannLabs/alphatims).

In [14]:
bruker_file_path = '../Data/PXD017703_diaPASEF/20200428_Evosep_60SPD_SG06-16_MLHeLa_200ng_py8_S3-A6_1_2452.d'

In [19]:
bruker_raw_data = alphatims.bruker.TimsTOF(file_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11872/11872 [00:25<00:00, 461.18it/s]


In [63]:
# extract the MS2 DIA spectrum from the frame 6005 (10.564 min) and m/z window: 400-425 Th
peptide_data = bruker_raw_data[6005, :, slice(400., 425.)]
peptide_data.head()

,raw_indices,frame_indices,scan_indices,precursor_indices,push_indices,tof_indices,rt_values,mobility_values,quad_low_mz_values,quad_high_mz_values,mz_values,intensity_values
0,279518033,6005,711,1,5519306,76458,633.841756,0.837191,400.0,425.0,248.158292,30
1,279518034,6005,711,1,5519306,144982,633.841756,0.837191,400.0,425.0,446.730501,59
2,279518035,6005,711,1,5519306,148134,633.841756,0.837191,400.0,425.0,457.258694,64
3,279518036,6005,711,1,5519306,158509,633.841756,0.837191,400.0,425.0,492.778941,74
4,279518037,6005,711,1,5519306,171814,633.841756,0.837191,400.0,425.0,540.274705,9


In [65]:
plot_mass_spectra(
    mz_values=peptide_data.mz_values.values, 
    intensity_values=peptide_data.intensity_values.values,
    title='MS2 DIA spectrum for the RT = 10.564 min and m/z window = 400-425 Th.'
).show(config=utils.config)

## Fig 2 G: Precursor/fragments elution profile 

For this figure we used the Bruker raw file is used from the [Project PXD017703 from ProteomeXchange](https://www.ebi.ac.uk/pride/archive/projects/PXD017703). as example data.

To read the raw TIMS-TOF data we use a recently published [AlphaTims package](https://github.com/MannLabs/alphatims).

An example is the following peptide:

> the information about the peptide was taken from the pyprophet_export_60SPD.tsv file.

* sequence: "IIIPEIQK"
* charge: 2
* m/z: 477.3051, Th
* im: 0.86, 1/K0
* rt: 703.802, seconds

![image](https://i.gyazo.com/a66e3676aa75dff9834d4acee7538b94.png)

In [374]:
peptide_info = {
    "sequence": "IIIPEIQK",
    "charge": 2,
    "mz": 477.3051,
    "im": 0.86,
    "rt": 703.802,
    "fragments": {
        "y8": 953.60304,
        "y7": 840.51897,
        "y6": 727.43491,
        "y5": 614.35085,
        "y4": 517.29808,
        "y3": 388.25549,
        "y2": 275.17143,
        "y1": 147.11285,
        "b1": 114.09139,
        "b2": 227.17545,
        "b3": 340.25951,
        "b4": 437.31228,
        "b5": 566.35487,
        "b6": 679.43893,
        "b7": 807.49751,
        "b8": 935.59247
    }      
}

In [375]:
# this plotting function is taken from the AlphaViz package (https://github.com/MannLabs/alphaviz) and modified

def plot_line(
    timstof_data,
    selected_indices,
    x_axis_label: str,
    label: str,
    y_axis_label: str = "intensity",
    remove_zeros: bool = False,
    trim: bool = True,
    width: int = 900,
    height: int = 400
):
    axis_dict = {
        "mz": "m/z, Th",
        "rt": "RT, min",
        "mobility": "Inversed IM, V·s·cm\u207B\u00B2",
        "intensity": "Intensity",
    }
    x_axis_label = axis_dict[x_axis_label]
    y_axis_label = axis_dict[y_axis_label]
    labels = {
        'm/z, Th': "mz_values",
        'RT, min': "rt_values",
        'Inversed IM, V·s·cm\u207B\u00B2': "mobility_values",
    }
    x_dimension = labels[x_axis_label]
    intensities = timstof_data.bin_intensities(selected_indices, [x_dimension])

    if x_dimension == "mz_values":
        x_ticks = timstof_data.mz_values
        plot_title = f"Spectrum"
    elif x_dimension == "mobility_values":
        x_ticks = timstof_data.mobility_values
        plot_title = f"Mobilogram"
    elif x_dimension == "rt_values":
        x_ticks = timstof_data.rt_values
        plot_title = f"XIC"
    non_zeros = np.flatnonzero(intensities)
    if len(non_zeros) == 0:
        x_ticks = np.empty(0, dtype=x_ticks.dtype)
        intensities = np.empty(0, dtype=intensities.dtype)
    else:
        if remove_zeros:
            x_ticks = x_ticks[non_zeros]
            intensities = intensities[non_zeros]
        elif trim:
            start = max(0, non_zeros[0] - 1)
            end = non_zeros[-1] + 2
            x_ticks = x_ticks[start: end]
            intensities = intensities[start: end]

    fig = go.Scatter(
        x=x_ticks,
        y=intensities,
        mode='lines',
        text = [f'{x_axis_label}'.format(i + 1) for i in range(len(x_ticks))],
        hovertemplate='<b>%{text}:</b> %{x};<br><b>Intensity:</b> %{y}.',
        name=label,
        line_shape='spline',
        line_smoothing=0
    )
    return fig

In [376]:
def plot_elution_profile(
    timstof_data,
    selected_indices,
    peptide_info,
    mz_tol: int = 50,
    rt_tol: int = 30,
    im_tol: int = 0.06,
    x_axis_label: str = "rt",
    y_axis_label: str = "intensity",
    title: str = "",
    width: int = 900,
    height: int = 400
):
    fig = go.Figure()
    
    rt_slice = slice(peptide_info['rt'] - rt_tol, peptide_info['rt'] + rt_tol)
    im_slice = slice(peptide_info['im'] - im_tol, peptide_info['im'] + im_tol)
#     int_slice = slice(100, None)
    
    # create an elution profile for the precursor
    precursor_indices = timstof_data[
        {"frame_indices": rt_slice,
         "scan_indices": im_slice,
         "mz_values": slice(peptide_info['mz'] / (1 + mz_tol / 10**6), peptide_info['mz'] * (1 + mz_tol / 10**6)),
#          "intensity_values": int_slice
        },
        'raw'
    ]
    fig.add_trace(
        plot_line(bruker_raw_data, precursor_indices, x_axis_label='rt', remove_zeros=True, label = 'precursor')
    )
    
    for frag, frag_mz in peptide_info['fragments'].items():
        
        fragment_data_indices = bruker_raw_data[{
            "frame_indices": rt_slice,
            "scan_indices": im_slice,
            "mz_values": slice(frag_mz / (1 + mz_tol / 10**6), frag_mz * (1 + mz_tol / 10**6)),
#             "intensity_values": int_slice, 
            },
        'raw']
        if len(fragment_data_indices) > 0:
            fig.add_trace(
                plot_line(bruker_raw_data, fragment_data_indices, x_axis_label='rt', remove_zeros=True, label = frag)
            )
    
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(
                size=16,
            ),
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        xaxis=dict(
            title=x_axis_label,
            titlefont_size=14,
            tickmode = 'auto',
            tickfont_size=14,
        ),
        yaxis=dict(
            title=y_axis_label
        ),
        template = "plotly_white", 
        width=width,
        height=height,
        hovermode="x",
        showlegend=True
    )
    return fig

In [377]:
plot_elution_profile(
    bruker_raw_data, 
    fragment_data_indices, 
    peptide_info,
    title='Precursor/fragments elution profile'
)